In [26]:
# Copied from: https://medium.com/python-data/effient-frontier-in-python-34b0c3043314
# This program was created for my own educational purposes to truly understand Markowitz Portfolio Theory
# No part of this program is used for commercial purposes

import quandl
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

quandl.ApiConfig.api_key = '3qNsf3VUNsvytS7sS_7W'
selected = ['TSLA', 'FB', 'MSFT', 'GE', 'AAPL']
data = quandl.get_table('WIKI/PRICES', ticker = selected, qopts = { 'columns': ['date', 'ticker', 'adj_close'] }, date = { 'gte': '2018-1-1', 'lte': '2018-12-31' }, paginate=True)

clean = data.set_index('date')
table = clean.pivot(columns='ticker')

print(table.head())

port_weight = [0.25, 0.35, 0.10, 0.20, 0.10]
table = table.mul(port_weight, axis=1)

print(table.head())

table = table.sum(axis=1)

print(table.head())

# Calculate daily & annual returns of the stocks
returns_daily = table.pct_change()
# Why does this * 253 not (1 + r)^253?
returns_annual = returns_daily.mean() * 253

# Calculate daily & annual covariance
cov_daily = returns_daily.cov()
cov_annual = cov_daily * 253

# Lists to store portfolio attributes
port_returns = []
port_volatility = []
stock_weight = []

# set the number of combinations for imaginary portfolios
num_assets = len(selected)
num_portfolio = 10000 # Number of imaginary portfolios

# Populate empty lists with each port's attributes
for port in range(num_portfolio):
    weights = np.random.random(num_assets)
    weights /= np.sum(weights)
    returns = np.dot(weights, returns_annual)
    volatility = np.sqrt(np.dot(weights.T, np.dot(cov_annual, weights)))
    port_returns.append(returns)
    port_volatility.append(volatility)
    stock_weight.append(weights)
    
portfolio = {
    'Returns': port_returns,
    'Volatility': port_volatility
}

# print(np.dot(weights.T, np.dot(cov_annual, weights)))

# extend portfolio to accomodate ticker & weight
for counter, symbol in enumerate(selected):
    portfolio[symbol + ' Weight'] = [Weight[counter] for Weight in stock_weight]

df = pd.DataFrame(portfolio)

column_order = ['Returns', 'Volatility'] + [stock + ' Weight' for stock in selected]

df = df[column_order]

# plt.style.use('seaborn')
# df.plot.scatter(x='Volatility', y='Returns', grid=True)
# plt.xlabel('Volatility')
# plt.ylabel('Expected Returns')
# plt.title('Efficient Frontier')
# plt.show()

           adj_close                              
ticker          AAPL      FB     GE   MSFT    TSLA
date                                              
2018-01-02    172.26  181.42  17.98  85.95  320.53
2018-01-03    172.23  184.67  18.15  86.35  317.25
2018-01-04    173.03  184.33  18.53  87.11  314.62
2018-01-05    175.00  186.85  18.54  88.19  316.58
2018-01-08    174.35  188.28  18.28  88.28  336.41
           adj_close                                
ticker          AAPL       FB     GE    MSFT    TSLA
date                                                
2018-01-02   43.0650  63.4970  1.798  17.190  32.053
2018-01-03   43.0575  64.6345  1.815  17.270  31.725
2018-01-04   43.2575  64.5155  1.853  17.422  31.462
2018-01-05   43.7500  65.3975  1.854  17.638  31.658
2018-01-08   43.5875  65.8980  1.828  17.656  33.641
date
2018-01-02    157.6030
2018-01-03    158.5020
2018-01-04    158.5100
2018-01-05    160.2975
2018-01-08    162.6105
dtype: float64


TypeError: cov() missing 1 required positional argument: 'other'